In [1]:
print('hi')

hi


In [2]:
import gym
from gym import spaces
from rljax.algorithm import DQN
from rljax.trainer import Trainer
import numpy as np
import pandas as pd

from environment import Env

In [3]:
data = pd.read_csv("SH_SDS_data_4.csv", index_col=0)
data.current_state = [('000'+str(cs))[-3:] for cs in data.current_state]
prob = pd.read_csv("SH_SDS_transition_matrix.csv", index_col=0)
prob.index = [('000'+str(idx))[-3:] for idx in prob.index]


In [4]:
def sniper(state_1, state_2, env, start_state):
    state = start_state
    done = False
    all_rewards = list()
    while not done:
        if state[0] == state_1:
            state, reward, done, _ = env.step(0)
        elif state[0] == state_2:
            state, reward, done, _ = env.step(1)
        else:
            state, reward, done, _ = env.step(2)
        all_rewards.append(reward)
    return all_rewards

In [5]:
env = Env(data, prob, steps=2000)
mappings = env.mapping
sniper(mappings['121'], mappings['401'], env, env.reset())
sniper(mappings['122'], mappings['401'], env, env.reset())
env.collapse_num_trades_dict

{'401': array([-24,  24]), '121': array([ 23, -23])}
{'401': array([-10,  10]), '122': array([ 9, -9])}


{'401': array([-34,  34]), '121': array([ 23, -23]), '122': array([ 9, -9])}

In [6]:
env.num_trades


[{'401': array([-34,  34]), '121': array([ 23, -23]), '122': array([ 9, -9])},
 {'401': array([-24,  24]), '121': array([ 23, -23])},
 {'401': array([-10,  10]), '122': array([ 9, -9])}]

In [ ]:
# 23,400 seconds between 9:30am and 4pm broken in 10 second increments
# Stepping through 50,000 trading days

NUM_AGENT_STEPS = 50000
SEED = 0

env = Env(data, prob, steps=2340)
env_test = Env(data, prob,steps=2340)

algo = DQN(
    num_agent_steps=NUM_AGENT_STEPS,
    state_space=env.state_space,
    action_space=env.action_space,
    seed=SEED,
    batch_size=256,
    start_steps=1000,
    update_interval=1,
    update_interval_target=400,
    eps_decay_steps=0,
    loss_type="l2",
    lr=1e-3,
)

trainer = Trainer(
    env=env,
    env_test=env_test,
    algo=algo,
    log_dir="",
    num_agent_steps=NUM_AGENT_STEPS,
    eval_interval=1000,
    seed=SEED,
)
trainer.train()

Num steps: 1000     Return: -48601.1   Time: 0:00:50
Num steps: 2000     Return: -29274.0   Time: 0:01:53
Num steps: 3000     Return: -26928.9   Time: 0:02:44
Num steps: 4000     Return: -25343.7   Time: 0:03:37
Num steps: 5000     Return: -30975.7   Time: 0:04:32
Num steps: 6000     Return: -20109.6   Time: 0:05:24
Num steps: 7000     Return: -37934.0   Time: 0:06:16
Num steps: 8000     Return: -28328.9   Time: 0:07:09
Num steps: 9000     Return: -32403.0   Time: 0:07:56
Num steps: 10000    Return: -30814.4   Time: 0:08:50
Num steps: 11000    Return: -34539.8   Time: 0:09:49
Num steps: 12000    Return: -26634.3   Time: 0:11:09
Num steps: 13000    Return: -33446.8   Time: 0:12:01


In [ ]:
env.plot()

In [ ]:
env.plot('share_history')